In [1]:
import torch
import mujoco
import mujoco.viewer
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from agent import SACAgent
from utils import ReplayBuffer
from trainer import SACTrainer

# This solve the pendulum environment with soft actor critic algorithm

# It uses a bunch of tricks to make this work better:
# Twin Q-Networks
# Memory Replay (Experience Replay Buffer)
# Target Networks with Polyak Averaging
# Automatic Entropy Tuning
# Reparameterization Trick
# Tanh Action Squashing with Log-Probability Correction
# Gradient Clipping
# Random Action Initialization

In [ ]:
class SequentialReachingEnv(gym.Env):
    """Custom 2-Joint Limb with 4 Muscles, 12 Sensors, and a Target Position"""

    def __init__(
        self,
        xml_path="path/to/your_model.xml",
        max_num_targets=10,
        max_target_duration=3,
    ):
        super().__init__()

        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)
        self.max_num_targets = max_num_targets
        self.max_target_duration = max_target_duration
        self.viewer = None

        num_actuators = self.model.nu

        # Define bounds for each sensor
        x_low, x_high = -0.7, 0.9
        y_low, y_high = -0.9, 0.4
        len_low, len_high = .1, .5
        vel_low, vel_high = -1, 1
        frc_low, frc_high = -100, 0

        # Observation space: 12 sensor readings + 3D target position
        self.observation_space = gym.spaces.Box(
            low=np.concatenate(
                [
                    [x_low, 0, y_low],  # Target position
                    np.full(num_actuators, len_low),  # Actuator positions
                    np.full(num_actuators, vel_low),  # Actuator velocities
                    np.full(num_actuators, frc_low),  # Actuator forces
                ]
            ),
            high=np.concatenate(
                [
                    [x_high, 0, y_high],
                    np.full(num_actuators, len_high),
                    np.full(num_actuators, vel_high),
                    np.full(num_actuators, frc_high),
                ]
            ),
            dtype=np.float32,
        )

        # Action space: 4 muscle activations
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(4,), dtype=np.float32)

        # Get the site ID using the name of your end effector
        self.hand_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_SITE, "hand")

        # Parse target positions from CSV file
        self.reachable_positions = self.parse_targets("../targets.csv")

    def parse_targets(self, targets_path="path/to/targets.csv", bins=100):
        target_positions = np.loadtxt(
            targets_path, delimiter=",", skiprows=1, usecols=(0, 2)
        )
        x, y = target_positions[:, 0], target_positions[:, 1]
        counts2d, x_edges, y_edges = np.histogram2d(x, y, bins=bins)
        x_centers = (x_edges[:-1] + x_edges[1:]) / 2
        y_centers = (y_edges[:-1] + y_edges[1:]) / 2
        nonzero_indices = np.argwhere(counts2d > 0)
        return [(x_centers[i], 0, y_centers[j]) for i, j in nonzero_indices]

    def sample_targets(self, num_samples=10):
        sampled_positions = np.random.choice(
            len(self.reachable_positions), num_samples, replace=False
        )
        return [self.reachable_positions[i] for i in sampled_positions]

    def update_target(self, position):
        self.data.mocap_pos = position
        mujoco.mj_forward(self.model, self.data)

    def step(self, action):
        self.data.ctrl[:] = action
        mujoco.mj_step(self.model, self.data)

        sensor_data = self.data.sensordata.copy()
        hand_position = self.data.site_xpos[self.hand_id]
        distance = np.linalg.norm(
            hand_position - self.target_positions[self.target_idx]
        )
        reward = -distance

        done = self.data.time > self.max_target_duration * self.max_num_targets
        terminated = False

        # doesn't make sense for learning
        if distance < .05: # self.data.time > self.max_target_duration * (self.target_idx + 1):
            terminated = True
            # reward += 1

            if self.target_idx < self.max_num_targets - 1:
                self.target_idx += 1
                self.update_target(self.target_positions[self.target_idx])
            else:
                done = True

        obs = np.concatenate([self.target_positions[self.target_idx], sensor_data])
        return obs, reward, done, terminated, {}

    def reset(self, seed=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)

        self.target_positions = self.sample_targets(self.max_num_targets)
        self.target_idx = 0
        self.update_target(self.target_positions[self.target_idx])

        sensor_data = self.data.sensordata.copy()
        obs = np.concatenate([self.target_positions[self.target_idx], sensor_data])
        return obs, {}

    def render(self):
        if self.viewer is not None:
            self.viewer.sync()
        else:
            self.viewer = mujoco.viewer.launch_passive(self.model, self.data)
            self.viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True
            self.viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_ACTUATOR] = True
            self.viewer.cam.lookat[:] = [0, -1.5, -0.5]
            self.viewer.cam.azimuth = 90
            self.viewer.cam.elevation = 0

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None

In [4]:
# env = gym.make("Pendulum-v1", render_mode="human")
env = SequentialReachingEnv(
    xml_path="../arm_model.xml",
    max_num_targets=1,
    max_target_duration=3,
)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

print("State dimension:", state_dim)
print("Action dimension:", action_dim)

hidden_layers = [256, 256]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device:", device)

c:\Users\User\anaconda3\envs\animat\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Users\User\anaconda3\envs\animat\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


State dimension: 15
Action dimension: 4
Device: cuda


In [ ]:
# Create SAC agent, replay buffer, and trainer
agent = SACAgent(state_dim, action_dim, hidden_layers, device=device)
replay_buffer = ReplayBuffer(state_dim, action_dim)
trainer = SACTrainer(
    env,
    agent,
    replay_buffer,
    batch_size=256,
    start_steps=1000,
    update_after=1000,
    update_every=50,
    max_episode_steps=200,
)

# Run training for a specified number of episodes
trainer.run(num_episodes=200)

Episode: 001 | Reward: -83.42
Episode: 002 | Reward: -79.05
Episode: 003 | Reward: -158.50
Episode: 004 | Reward: -138.00
Episode: 005 | Reward: -86.20
Episode: 006 | Reward: -75.55
Episode: 007 | Reward: -165.35
Episode: 008 | Reward: -159.99
Episode: 009 | Reward: -158.60
Episode: 010 | Reward: -128.12
Episode: 011 | Reward: -98.95
Episode: 012 | Reward: -158.03
Episode: 013 | Reward: -158.62
Episode: 014 | Reward: -93.27
Episode: 015 | Reward: -85.22
Episode: 016 | Reward: -86.08
Episode: 017 | Reward: -61.37
Episode: 018 | Reward: -108.56
Episode: 019 | Reward: -142.41
Episode: 020 | Reward: -94.86
Episode: 021 | Reward: -90.73
Episode: 022 | Reward: -182.50
Episode: 023 | Reward: -157.06
Episode: 024 | Reward: -47.60
Episode: 025 | Reward: -103.26
Episode: 026 | Reward: -89.68
Episode: 027 | Reward: -145.00
Episode: 028 | Reward: -173.19
Episode: 029 | Reward: -59.75
Episode: 030 | Reward: -40.50
Episode: 031 | Reward: -130.03
Episode: 032 | Reward: -142.30


In [9]:
env = gym.make("Pendulum-v1", render_mode="human")

# uncomment below if you want to visualize the result of the training
state, _ = env.reset()
done = False
while not done:
   env.render()  # Renders the environment window (ensure you have a display)
   action = agent.select_action(state, deterministic=False)
   next_state, reward, terminated, truncated, info = env.step(action)
   done = terminated or truncated
   state = next_state
env.close()